Use a CNN to classify trimmed spectra


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

## Import data

In [2]:
# Read in individual datacubes and combine into a single dataframe

from astropy.table import Table
import pandas as pd

df = pd.DataFrame()

sightline_list = ['J010619+004823','J012403+004432','J013340+040059','J013724-422417','J015741-010629',
                  'J020944+051713','J024401-013403','J033413-161205','J033900-013318','J094932+033531',
                  'J095852+120245','J102009+104002','J111008+024458','J111113-080401','J120917+113830',
                  'J123055-113909','J124957-015928','J133254+005250','J142438+225600','J162116-004251',
                  'J193957-100241','J200324-325145','J205344-354652','J221527-161133','J230301-093930',
                  'J231543+145606','J233446-090812','J234913-371259']

for sightline in sightline_list:
    table = Table.read('/home/emma/Documents/laeML/data/catalogues/individual_cubes/{}_cubex_data.fits'.format(sightline))
    table['sightline'] = sightline
    pandas_df = table.to_pandas()
    df = pd.concat([df,pandas_df])

In [3]:
# Cut to SNR > 7 as we haven't looked at all the sources below this
df = df[df.SNR >= 7]
df = df.reset_index(drop=True)


In [4]:
# Check number of LAEs
type_str = df['type'].str.decode("utf-8")
nLAE = type_str.str.contains('LAE').sum()
print(nLAE)

1013


In [5]:
df.type.value_counts()

b'None'        30060
b'LAE'           971
b'QSO'           272
b'OII'           240
b'[OII]'          41
b'LAE?'           29
b'OIII'           22
b'CIV'            16
b'[OIII]'         11
b'LAE??'           9
b'EL'              9
b'CIII]'           6
b'CIII'            6
b'Ha'              5
b'LAEb'            3
b'Hb'              2
b'Hg'              2
b'OIII?'           2
b'LAE??OII'        1
b'NII'             1
b'Hbeta'           1
b'Ha?'             1
b'Nebula'          1
b'CIV?'            1
b'Star'            1
b'OII?'            1
b'[CIII]'          1
Name: type, dtype: int64

In [6]:
# Create new column to show if entry is an LAE
df['isLAE'] = np.where(type_str.str.contains('LAE'), 1, 0)
df['isOxy'] = np.where(type_str.str.contains('OII'), 1, 0)

In [7]:
df.isOxy[df.isOxy==1]

58       1
72       1
135      1
183      1
310      1
        ..
31544    1
31548    1
31614    1
31684    1
31691    1
Name: isOxy, Length: 318, dtype: int64

We want to use 1D flux arrays from the trimmed spectra as input to our CNN (i.e. X) while is_LAE is the target (i.e. Y)

In [8]:
y=df.isLAE

Need to read in the flux columns from the trimmed arrays and combine into a dataframe to use as X

In [9]:
#instead of creating x from each file individually, read it in from a prepared file
xAll=pd.read_csv("/home/emma/Documents/laeML/data/trimmed_spectra_dataframe.csv",index_col=0)

In [10]:
xAll.head()

,0,1,2,3,4,5,6,7,8,9,...,152,153,154,155,156,157,158,159,160,161
0,J010619+004823,5,590.384766,454.941467,354.345154,328.243042,5.348419,70.211662,235.562622,207.469299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,J010619+004823,9,452.941223,427.720490,435.023926,538.362366,448.548431,518.894287,402.824951,319.419006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,J010619+004823,18,237.862305,131.504059,196.990021,375.919373,264.392822,225.224884,189.368408,158.257416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,J010619+004823,703,371.443054,337.741241,242.179245,256.283417,389.117188,477.863403,372.696228,448.540527,...,290.749451,296.189941,207.545380,299.615936,240.926071,307.433838,527.028931,481.803894,500.537750,474.636536
4,J010619+004823,784,47.715321,-27.427366,42.573402,34.838043,89.349220,-45.284454,-17.226364,-45.073406,...,114.334274,35.456001,-106.346481,25.491888,45.191193,45.270767,-9.925606,-58.586670,39.541794,29.653870


In [11]:
#remove rows identified as OII for now to avoid confusion
y = y[df.isOxy != 1]
xAll = xAll[df.isOxy != 1]

In [12]:
#drop info columns with sightline and id to leave just the flux array
x=xAll.drop(['0','1'], axis=1)

In [13]:
#check correct columns are left
x.head()

,2,3,4,5,6,7,8,9,10,11,...,152,153,154,155,156,157,158,159,160,161
0,590.384766,454.941467,354.345154,328.243042,5.348419,70.211662,235.562622,207.469299,137.152069,109.381187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,452.941223,427.720490,435.023926,538.362366,448.548431,518.894287,402.824951,319.419006,319.533020,267.410187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,237.862305,131.504059,196.990021,375.919373,264.392822,225.224884,189.368408,158.257416,97.674271,99.918762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,371.443054,337.741241,242.179245,256.283417,389.117188,477.863403,372.696228,448.540527,375.683411,559.601074,...,290.749451,296.189941,207.545380,299.615936,240.926071,307.433838,527.028931,481.803894,500.537750,474.636536
4,47.715321,-27.427366,42.573402,34.838043,89.349220,-45.284454,-17.226364,-45.073406,23.561562,-54.283142,...,114.334274,35.456001,-106.346481,25.491888,45.191193,45.270767,-9.925606,-58.586670,39.541794,29.653870


## Pre-process data

### Normalise spectra

Rescale spectra so that they are normalised and continuum is the same for all sources

In [14]:
for row in range(len(x)):
    
    #rescale continuum
    contavg = np.nanmedian(x.iloc[row])
    x.iloc[row] = x.iloc[row] - contavg
    
    #normalise peak to 1
    peakmax = np.max(x.iloc[row][50:100])
    x.iloc[row] = x.iloc[row]/peakmax 

In [15]:
#replace Nans with zero
x=x.fillna(0)

In [16]:
x = np.array(x).reshape(x.shape[0], x.shape[1], 1)
print(x.shape)

(31397, 160, 1)


In [17]:
#create the final test/validation sample that the model never sees
from sklearn.model_selection import train_test_split
x, x_final, y, y_final=train_test_split(x, y, test_size=0.1)

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.15)

In [19]:
nLAE_trainSample = len(y_train[y_train==1])
print(nLAE_trainSample)

787


## Create simple CNN

In [20]:
#import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 2, activation="relu", input_shape=(160,1)))
model.add(MaxPooling1D())
model.add(Conv1D(64, 2, activation="relu"))
model.add(MaxPooling1D())
model.add(Dense(16, activation="relu"))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))


In [21]:
model.compile(loss = 'binary_crossentropy', 
     optimizer = "adam",               
    metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 159, 64)           192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 79, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 78, 64)            8256      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 39, 64)           0         
 1D)                                                             
                                                                 
 dense (Dense)               (None, 39, 16)            1040      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 19, 16)           0

In [22]:
model.fit(x_train, y_train,
          batch_size=200,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/20
121/121 [==============================] - 5s 36ms/step - loss: 0.2102 - accuracy: 0.9655 - val_loss: 0.1446 - val_accuracy: 0.9684
Epoch 2/20
121/121 [==============================] - 4s 32ms/step - loss: 0.1397 - accuracy: 0.9672 - val_loss: 0.1246 - val_accuracy: 0.9684
Epoch 3/20
121/121 [==============================] - 6s 47ms/step - loss: 0.1167 - accuracy: 0.9671 - val_loss: 0.1059 - val_accuracy: 0.9684
Epoch 4/20
121/121 [==============================] - 6s 50ms/step - loss: 0.1018 - accuracy: 0.9671 - val_loss: 0.0978 - val_accuracy: 0.9684
Epoch 5/20
121/121 [==============================] - 5s 45ms/step - loss: 0.0945 - accuracy: 0.9671 - val_loss: 0.0934 - val_accuracy: 0.9689
Epoch 6/20
121/121 [==============================] - 5s 41ms/step - loss: 0.0909 - accuracy: 0.9680 - val_loss: 0.0931 - val_accuracy: 0.9677
Epoch 7/20
121/121 [==============================] - 5s 41ms/step - loss: 0.0885 - accuracy: 0.9687 - val_loss: 0.0909 - val_accuracy: 0.9684

In [23]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0874500647187233
Test accuracy: 0.9683887958526611


In [24]:
ypred = model.predict(x_final)
ypred = np.array(ypred)

for yi in range(len(ypred)):
    if ypred[yi]<0.5:
        ypred[yi] = 0
    else:
        ypred[yi] = 1

99/99 [==============================] - 0s 3ms/step


In [25]:
y_final=np.array(y_final)
nLAE_finalsample = len(y_final[y_final==1])
print(nLAE_finalsample)

91


In [26]:
isLAE_and_predLAE,isLAE_and_NotpredLAE,NotLAE_and_predLAE, NotLAE_and_NotpredLAE = 0,0,0,0

for i in range(len(y_final)):
    if (y_final[i] == 1) & (ypred[i][0]==1):
        isLAE_and_predLAE += 1 # Number of True LAEs predicted to be LAEs
    elif (y_final[i] == 1) & (ypred[i][0]!=1):
        isLAE_and_NotpredLAE += 1 # Number of True LAEs not predicted to be LAEs
    if (y_final[i] != 1) & (ypred[i][0]==1):
        NotLAE_and_predLAE += 1 # Number of non-LAEs predicted to be LAEs
    if (y_final[i] != 1) & (ypred[i][0] != 1):
        NotLAE_and_NotpredLAE += 1 # Number of non-LAEs predicted to not be LAEs

In [27]:
print(isLAE_and_predLAE,isLAE_and_NotpredLAE,NotLAE_and_predLAE,NotLAE_and_NotpredLAE)

19 72 8 3041


In [28]:
19/91.


0.2087912087912088